In [1]:
import torch
import torchvision

import sys
import os
import argparse
import warnings
import numpy as np
#!pip install pandas
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout


from sklearn.neighbors import KernelDensity
from keras.models import load_model

2024-12-31 11:57:03.523064: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(torch.__version__)
file_path = "../data/nsl/"

2.2.2


### Data Preprocessing

In [31]:
df = pd.read_csv(file_path+"KDDTest+.txt", header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [20]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])


In [32]:

df.columns = columns
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [33]:
def get_unique(df, column_name = 'attack'):
    unique_vals = df[column_name].unique()
    value_counts = df[column_name].value_counts()
    print(f"Unique Values ({len(unique_vals)}): {unique_vals}\n")
    print(f"Value Counts:\n{value_counts}\n{'='*40}\n")
get_unique(df)

Unique Values (38): ['neptune' 'normal' 'saint' 'mscan' 'guess_passwd' 'smurf' 'apache2'
 'satan' 'buffer_overflow' 'back' 'warezmaster' 'snmpgetattack'
 'processtable' 'pod' 'httptunnel' 'nmap' 'ps' 'snmpguess' 'ipsweep'
 'mailbomb' 'portsweep' 'multihop' 'named' 'sendmail' 'loadmodule' 'xterm'
 'worm' 'teardrop' 'rootkit' 'xlock' 'perl' 'land' 'xsnoop' 'sqlattack'
 'ftp_write' 'imap' 'udpstorm' 'phf']

Value Counts:
attack
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail           

In [34]:
attack_n = []
for i in df.attack :
  if i == 'normal':
    attack_n.append("normal")
  else:
    attack_n.append("attack")
df['attack'] = attack_n

get_unique(df)

Unique Values (2): ['attack' 'normal']

Value Counts:
attack
attack    12833
normal     9711
Name: count, dtype: int64



In [35]:
cat_features = df.select_dtypes(include='object').columns
cat_features

Index(['protocol_type', 'service', 'flag', 'attack'], dtype='object')

In [5]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
#clm=['protocol_type', 'service', 'flag', 'attack']
for x in cat_features:
    print(x)
    df[x]=le.fit_transform(df[x])

NameError: name 'cat_features' is not defined

In [42]:
# Save the DataFrame to a CSV file
df.to_csv('data/nsl/test.csv', index=False)

In [43]:
df.shape

(22544, 43)

In [2]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('data/nsl/test.csv')
y = df["attack"]
X = df.drop(["attack"], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, y,test_size=0.1,random_state=43) 

FileNotFoundError: [Errno 2] No such file or directory: 'data/nsl/test.csv'

In [7]:
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
df = pd.read_csv('data/nsl/test.csv')
y = df["attack"]
X = df.drop(["attack"], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, y,test_size=0.1,random_state=43) 
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # we use only transform in order to prevent data leakage

FileNotFoundError: [Errno 2] No such file or directory: 'data/nsl/test.csv'

In [47]:
X_train[0].shape

(42,)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(64, input_dim=42, activation='relu'),
    Dense(128, activation='relu'),
    # Dropout(0.5),  # Uncomment for dropout
    Dense(64, activation='relu'),
    # Dropout(0.5),  # Uncomment for dropout
    Dense(1, activation='sigmoid')  # For binary classification
])

2024-12-15 19:07:58.789467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [50]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9489 - loss: 0.1538 - val_accuracy: 0.9911 - val_loss: 0.0316
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9916 - loss: 0.0251 - val_accuracy: 0.9947 - val_loss: 0.0215
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9942 - loss: 0.0204 - val_accuracy: 0.9947 - val_loss: 0.0178
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9953 - loss: 0.0144 - val_accuracy: 0.9956 - val_loss: 0.0159
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9937 - loss: 0.0158 - val_accuracy: 0.9951 - val_loss: 0.0162
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9957 - loss: 0.0131 - val_accuracy: 0.9951 - val_loss: 0.0179
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9956 - loss: 0.0128 - val_accuracy: 0.9956 - val_loss: 0.0175
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9948 - loss: 0.0156 - val_accuracy: 0.

In [ ]:
print("Y_test shape: ", y_test.shape)

Y_test shape:  (2255,)


### Detect AE

In [3]:
import os
import argparse
import warnings
import numpy as np
from sklearn.neighbors import KernelDensity
from keras.models import load_model
import sys
sys.path.append('../')  # Add the parent directory to the Python path
from util import *

In [50]:
class NSLModel(torch.nn.Module):
   def __init__(self):
       super(NSLModel, self).__init__()
       self.fc1 = torch.nn.Linear(121, 50)
       self.fc2 = torch.nn.Linear(50, 2)

   def forward(self, x):
       x = torch.relu(self.fc1(x))
       x = torch.softmax(self.fc2(x), dim=1)
       return x
   
def create_nsl_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(50, activation='relu', input_shape=(121,)),  # Input layer
        tf.keras.layers.Dense(2, activation='softmax')  # Output layer
    ])
    return model

In [69]:

# Optimal KDE bandwidths that were determined from CV tuning
BANDWIDTHS = {'mnist': 1.20, 'cifar': 0.26, 'svhn': 1.00, 'nsl': 1.00, 'c17':1.00}
class ArgsClass:
    def __init__(self):
        self.dataset = 'nsl' # 'c17' #
        self.attack = 'diff' #'pgd' #
        #self.dataset = 'nsl'
        self.batch_size = 256
        #self.dataset = 'nsl'

    def my_method(self):
        print(self.dataset)

args = ArgsClass()
args.dataset

'nsl'

In [70]:
if args.dataset == 'nsl':
    folder_path = "NSLKDD/"
    #model = load_model('../data/model_nsl.h5')#correct
    model = create_nsl_model()
    model.load_weights('NSLKDD/Models/nsl_kdd_Dos.h5')
else:
    folder_path = "CICIDS2017/"
    model = load_model('CICIDS2017/Models/cicids_binary.h5') #correct

data_path = folder_path + "OriginalData/" #/manda/NSLKDD/OriginalData/X_test.npy

# Load the dataset
X_train, y_train = np.load(data_path+'X_train.npy'), np.load(data_path+'y_train.npy')
X_test, y_test =  np.load(data_path+'X_test.npy'), np.load(data_path+'y_test.npy')
print("Original X_train shape:", X_train.shape, "  Original y_train shape:", y_train.shape )
print("Original X_test shape:", X_test.shape, "  Original Y_test shape:", y_test.shape)

#assert os.path.isfile('NSLKDD/Models/nsl_kdd_Dos.h5'), 'model file not found.'

Original X_train shape: (108270, 121)   Original y_train shape: (108270,)
Original X_test shape: (17169, 121)   Original Y_test shape: (17169,)


/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [71]:
if args.attack == 'diff':
    adv_path = folder_path + "DiffAttack/" 
elif args.attack == 'pgd':
    adv_path = folder_path + "Synt_PGDAttack/" # "Org_PGDAttack/" # "Org_PGDAttack/" 
    
# Load adv
X_test_adv = np.load(adv_path + 'X_test.npy')
y_test_adv = np.load(adv_path + 'y_test.npy')
print("Original X_test_adv shape:", X_test_adv.shape)

Original X_test_adv shape: (15453, 121)


In [72]:
nClass = len(np.unique(y_train))
Y_train, Y_test =  to_categorical(y_train, nClass), to_categorical(y_test, nClass)
y_test_adv = to_categorical(y_test_adv, nClass) 
print(" Y_train shape:", Y_train.shape, "   Y_test shape:", Y_test.shape)

 Y_train shape: (108270, 2)    Y_test shape: (17169, 2)


In [ ]:

----artifacts
# Load the dataset
X_train, Y_train, X_test, Y_test = get_data(args.dataset)
print("Original X_test shape:", X_test.shape)
X_test_adv = np.load('../data/Adv_%s_%s.npy' % (args.dataset,
                                                        args.attack))
# Craft an equal number of noisy samples
X_test_noisy = get_noisy_samples(X_test, X_test_adv, args.dataset,
                                    args.attack)
print(type(X_test_adv))
print("Original X_test_adv shape:", X_test_adv.shape)

In [41]:
# Check model accuracies on each sample type
for s_type, dataset in zip(['normal', 'adversarial'], #['normal', 'noisy', 'adversarial']
                            [X_test, X_test_adv]): #[X_test, X_test_noisy, X_test_adv]
    _, acc = model.evaluate(dataset, Y_test, batch_size=args.batch_size,
                            verbose=0)
    print("Model accuracy on the %s test set: %0.2f%%" %
            (s_type, 100 * acc))
    # Compute and display average perturbation sizes
    if not s_type == 'normal':
        l2_diff = np.linalg.norm(
            dataset.reshape((len(X_test), -1)) -
            X_test.reshape((len(X_test), -1)),
            axis=1
        ).mean()
        print("Average L-2 perturbation size of the %s test set: %0.2f" %
                (s_type, l2_diff))

ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

In [74]:
print("clean data shape: ", clean_data.shape)
print("updated X_test_adv shape:", X_test_adv.shape)

clean data shape:  (15498, 121)
updated X_test_adv shape: (15453, 121)


In [73]:
import numpy as np
predictions = model.predict(X_test)
preds_test = np.argmax(predictions, axis=1)

inds_correct = np.where(preds_test == Y_test.argmax(axis=1))[0]

indexs = random_select(len(X_test), len(inds_correct))
clean_data = X_test[indexs]
print("clean data shape: ", clean_data.shape)

X_test = X_test[inds_correct]
#X_test_noisy = X_test_noisy[inds_correct]
X_test_adv = X_test_adv[inds_correct]
print("preds_test shape: ", preds_test.shape)
print("updated X_test_adv shape:", X_test_adv.shape)

537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
clean data shape:  (15498, 121)


IndexError: index 15453 is out of bounds for axis 0 with size 15453

In [ ]:
## Get Bayesian uncertainty scores
print('Getting Monte Carlo dropout variance predictions...')
uncerts_normal = get_mc_predictions(model, X_test, batch_size=args.batch_size).var(axis=0).mean(axis=1)
#uncerts_noisy = get_mc_predictions(model, X_test_noisy, batch_size=args.batch_size).var(axis=0).mean(axis=1)
uncerts_adv = get_mc_predictions(model, X_test_adv, batch_size=args.batch_size).var(axis=0).mean(axis=1)

uncerts_clean = get_mc_predictions(model, clean_data, batch_size=args.batch_size).var(axis=0).mean(axis=1)

Getting Monte Carlo dropout variance predictions...


100%|██████████| 50/50 [00:30<00:00,  1.64it/s]


In [49]:
## Get KDE scores
# Get deep feature representations
print('Getting deep feature representations...')
X_train_features = get_deep_representations(model, X_train, batch_size=args.batch_size, last_hidden_layer = -2)
X_test_normal_features = get_deep_representations(model, X_test, batch_size=args.batch_size, last_hidden_layer = -2)
#X_test_noisy_features = get_deep_representations(model, X_test_noisy, batch_size=args.batch_size)
X_test_adv_features = get_deep_representations(model, X_test_adv, batch_size=args.batch_size, last_hidden_layer = -2)
X_test_clean_features = get_deep_representations(model, clean_data, batch_size=args.batch_size, last_hidden_layer = -2)


Getting deep feature representations...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━

In [50]:
numClass = len(np.unique(Y_train))
print("--------------number of class label: ", numClass)
# Train one KDE per class
print('Training KDEs...')
class_inds = {}
for i in range(Y_train.shape[1]):
    class_inds[i] = np.where(Y_train.argmax(axis=1) == i)[0]

class_inds

--------------number of class label:  2
Training KDEs...


{0: array([    0,     3,     4, ..., 59993, 59997, 59998]),
 1: array([    1,     2,     5, ..., 59995, 59996, 59999])}

In [53]:
kdes = {}

for i in range(Y_train.shape[1]):
    kdes[i] = KernelDensity(kernel='gaussian',
                            bandwidth=BANDWIDTHS[args.dataset]) \
        .fit(X_train_features[class_inds[i]])

In [54]:
preds_test_normal = np.argmax(model.predict(X_test), axis=1)
#preds_test_noisy = np.argmax(model.predict(X_test_noisy), axis=1)
preds_test_adv = np.argmax(model.predict(X_test_adv), axis=1)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [55]:

preds_test_clean = np.argmax(model.predict(clean_data), axis=1)

densities_clean = score_samples(
    kdes,
    X_test_clean_features,
    preds_test_clean
)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


2024-12-30 19:05:38.317698: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:05:38.317703: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:05:38.317707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:05:38.317706: I tensorflow/core/platform/cpu_featu

In [56]:
# Get density estimates
print('computing densities...')
densities_normal = score_samples(
    kdes,
    X_test_normal_features,
    preds_test_normal
)
#densities_noisy = score_samples(kdes, X_test_noisy_features, preds_test_noisy)
densities_adv = score_samples(
    kdes,
    X_test_adv_features,
    preds_test_adv
)

computing densities...


2024-12-30 19:08:51.954787: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:08:51.954954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:08:51.955355: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 19:08:51.955882: I tensorflow/core/platform/cpu_featu

In [57]:
def normalize_std(normal, adv, clean):
    """
    TODO
    :param normal:
    :param adv:
    :param noisy:
    :return:
    """
    n_samples = len(normal)
    #scaler = StandardScaler()
    #total = scaler.fit_transform(np.concatenate((normal, adv, noisy, clean)))
    total = scale(np.concatenate((normal, adv, clean)))

    return total[:n_samples], total[n_samples:2*n_samples], total[2*n_samples:]


In [58]:
## Z-score the uncertainty and density values
uncerts_normal_z, uncerts_adv_z, uncerts_clean_z = normalize_std(
    uncerts_normal,
    uncerts_adv,
    #uncerts_noisy,
    uncerts_clean
)
densities_normal_z, densities_adv_z, densities_clean_z = normalize_std(
    densities_normal,
    densities_adv,
    #densities_noisy,
    densities_clean
)

In [65]:
uncerts_normal_z, uncerts_adv_z, uncerts_clean_z, densities_normal_z, densities_adv_z, densities_clean_z = \
        get_uncertainties_and_densities(model, X_train, Y_train, X_test, X_test_adv, clean_data,\
                                         bandwidth=BANDWIDTHS[args.dataset], batch_size=args.batch_size)

Getting -uncertainty- Monte Carlo dropout variance predictions...


100%|██████████| 50/50 [00:08<00:00,  5.90it/s]


Getting deep feature representations...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━

2024-12-31 15:10:19.825419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 15:10:19.825473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 15:10:19.825420: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 15:10:19.827723: I tensorflow/core/platform/cpu_featu

In [66]:
## Build detector
values, labels, lr = train_lr(
    densities_pos=densities_adv_z,
    densities_neg=densities_normal_z, #np.concatenate((densities_normal_z, densities_noisy_z)),
    uncerts_pos=uncerts_adv_z,
    uncerts_neg=uncerts_normal_z #np.concatenate((uncerts_normal_z, uncerts_noisy_z))
)

In [67]:
vals, labs = get_neg_values_for_lr(densities_clean_z, uncerts_clean_z)
acc = lr.score(vals, labs)
print("Model accuracy on the purified (remove detect adv) test set: %0.2f%%" % (100 * acc))


Model accuracy on the purified (remove detect adv) test set: 100.00%


In [68]:
## Evaluate detector
print('################### Detector Model: Overall Performance ###########################')
print_performance_metrics(lr, values, labels)
# Compute logistic regression model predictions
probs = lr.predict_proba(values)[:, 1]
# Compute AUC
n_samples = len(X_test)
# The first 2/3 of 'probs' is the negative class (normal and noisy samples),
# and the last 1/3 is the positive class (adversarial samples).
_, _, auc_score = compute_roc(
    probs_neg=probs[:n_samples], #[:2 * n_samples]
    probs_pos=probs[n_samples:] #[2 * n_samples:]]
)
print('Detector ROC-AUC score: %0.4f' % auc_score)


################### Detector Model: Overall Performance ###########################
Accuracy: 0.5297457736482126
Precision: 1.0
Recall: 0.05949154729642535
F1-score: 0.1123020706455542
Confusion Matrix:
[[15498     0]
 [14576   922]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.52      1.00      0.68     15498
         1.0       1.00      0.06      0.11     15498

    accuracy                           0.53     30996
   macro avg       0.76      0.53      0.40     30996
weighted avg       0.76      0.53      0.40     30996

Detector ROC-AUC score: 0.9865


In [78]:
fpr, tpr, _ = roc_curve(labels, probs)
print(fpr)
tpr

[0.00000000e+00 0.00000000e+00 2.62536099e-05 ... 4.46075085e-01
 4.46127593e-01 1.00000000e+00]


array([0.00000000e+00, 5.25072197e-05, 5.25072197e-05, ...,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00])

In [ ]:
y_pred = lr.predict(values)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [81]:
from sklearn import metrics
# Accuracy
accuracy = metrics.accuracy_score(labels, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6666666666666666


In [84]:
print('Detector Model Accuracy:: %0.2f' % (100*accuracy))

Detector Model Accuracy:: 66.67
